In [406]:
import pandas as pd
from sklearn.model_selection import train_test_split
import psycopg2 as psycopg2
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix 
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import time
from pathlib import Path

In [407]:
startTid = time.time()

In [408]:
testfile = Path("../Data/test.tsv")
trainfile = Path("../Data/train.tsv")
vaildfile = Path("../Data/valid.tsv")
# Code for reading the csv file and separating.
data = []
i = 0
breakpoint = 100
chunksize = 100
totalRow = breakpoint * chunksize

In [409]:
# Converting to a pandas dataframe.
dfTest = pd.read_table(testfile)
dfTrain = pd.read_table(trainfile)
dfValid = pd.read_table(vaildfile)

 
# Labeling the columns   
dfTest.columns = ['Id' ,'Label','Stament', 'Subject', 'Speaker', 'Job Title', 'State Info', 'Party', 'Brealy Truth', 'False Counts', 'Half Truth', 'Mostly Truth', 'Pants on Fire', 'Context']
dfTrain.columns = ['Id' ,'Label','Stament', 'Subject', 'Speaker', 'Job Title', 'State Info', 'Party', 'Brealy Truth', 'False Counts', 'Half Truth', 'Mostly Truth', 'Pants on Fire', 'Context']
dfValid.columns = ['Id' ,'Label','Stament', 'Subject', 'Speaker', 'Job Title', 'State Info', 'Party', 'Brealy Truth', 'False Counts', 'Half Truth', 'Mostly Truth', 'Pants on Fire', 'Context']

In [410]:
# Checking shapes 
print(dfTest.shape)
print(dfTrain.shape)
print(dfValid.shape)

# Merging the dataframes
df = pd.concat([dfTest,dfTrain,dfValid])
print (df.shape,df.head(0))

(1266, 14)
(10239, 14)
(1283, 14)
(12788, 14) Empty DataFrame
Columns: [Id, Label, Stament, Subject, Speaker, Job Title, State Info, Party, Brealy Truth, False Counts, Half Truth, Mostly Truth, Pants on Fire, Context]
Index: []


In [411]:

# Sorting our y variables into FAKE or REAL
df["Label"] = np.where(df["Label"] == "barely-true", "REAL", df["Label"])
df["Label"] = np.where(df["Label"] == "true", "REAL", df["Label"])
df['Label'] = np.where(df['Label'] == "mostly-true", "REAL", df["Label"])

df["Label"] = np.where(df["Label"] == "half-true", "FAKE", df["Label"])
df["Label"] = np.where(df["Label"] == "mostly-false", "FAKE", df["Label"])
df["Label"] = np.where(df["Label"] == "false", "FAKE", df["Label"])
df["Label"] = np.where(df["Label"] == "pants-fire", "FAKE", df["Label"])

In [412]:
baseline = df.iloc[:,1:3]
print(baseline.shape)
print ("Diller", baseline.head(2))

(12788, 2)
Diller   Label                                            Stament
0  FAKE  Wisconsin is on pace to double the number of l...
1  FAKE  Says John McCain has done nothing to help the ...


In [413]:
df2 = df.iloc[:,4:5]
print(df2.head(2))

             Speaker
0  katrina-shankland
1       donald-trump


In [414]:
frame = [baseline, df2]
baselineExtended = pd.concat(frame,  axis=1, join='inner') 
print (baselineExtended.head(2))

  Label                                            Stament            Speaker
0  FAKE  Wisconsin is on pace to double the number of l...  katrina-shankland
1  FAKE  Says John McCain has done nothing to help the ...       donald-trump


In [415]:
stemmer = PorterStemmer()   
words = stopwords.words("english")
Data_transformed = baselineExtended['Stament'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [416]:
# Creating a vectorized array with integers indicating frequency of words.
vectorize = CountVectorizer(max_features=200,ngram_range=(1,3))
x = vectorize.fit_transform(Data_transformed).toarray()
y = np.array(df['Label'])

In [417]:
# Creating a test split of 80/20.
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=34)
print('\nTotal number of samples in the Train Dataset :',xtrain.shape[0])
print('Total number of samples in the Test Dataset :',xtest.shape[0])


Total number of samples in the Train Dataset : 10230
Total number of samples in the Test Dataset : 2558


In [418]:
# Second baseline model using multinomial naive bayes.
mulNB = MultinomialNB().fit(xtrain, ytrain)
print('\nAccuracy score for Multinomial Naive Bayes :',mulNB.score(xtest,ytest))

ypred_mul = mulNB.predict(xtest)

print('Confusion Metrics for Multinomial Naive Bayes : \n\n',confusion_matrix(ytest, ypred_mul),'\n\n')
print('Classification Report for Multinomial Naive Bayes :\n\n',classification_report(ytest,ypred_mul))


Accuracy score for Multinomial Naive Bayes : 0.5723221266614542
Confusion Metrics for Multinomial Naive Bayes : 

 [[584 642]
 [452 880]] 


Classification Report for Multinomial Naive Bayes :

               precision    recall  f1-score   support

        FAKE       0.56      0.48      0.52      1226
        REAL       0.58      0.66      0.62      1332

    accuracy                           0.57      2558
   macro avg       0.57      0.57      0.57      2558
weighted avg       0.57      0.57      0.57      2558



In [419]:
# Third baseline model using Logistic regression.
scaler = StandardScaler()
lr = LogisticRegression(max_iter = 600)
log_model = Pipeline([('standardize', scaler),
                    ('log_reg', lr)])

log_model.fit(xtrain, ytrain)

ypred_log = log_model.predict(xtest)
test_accuracy = accuracy_score(ytest, ypred_log)*100

print('Testing accuracy for Logistic regression: %.4f %%' % test_accuracy) 
print('Confusion Metrics for Logistic regression:\n\n', confusion_matrix(ytest, ypred_log))
print('Classification Report for Multinomial Naive Bayes :\n\n',classification_report(ytest,ypred_log))

Testing accuracy for Logistic regression: 56.2549 %
Confusion Metrics for Logistic regression:

 [[577 649]
 [470 862]]
Classification Report for Multinomial Naive Bayes :

               precision    recall  f1-score   support

        FAKE       0.55      0.47      0.51      1226
        REAL       0.57      0.65      0.61      1332

    accuracy                           0.56      2558
   macro avg       0.56      0.56      0.56      2558
weighted avg       0.56      0.56      0.56      2558



In [420]:
slutTid = time.time()
kage2 = slutTid-startTid
print(kage2)
print("Tid i min:" ,kage2/60)

3.1255009174346924
Tid i min: 0.052091681957244874
